# Ontology Mapping Demo

This notebook demonstrates the ontology mapping process using different strategies.

In [1]:
%cd '/Users/abhilashdhal/Desktop/cbioportal_gsoc/OmicsMLRepoHarmonizer'

/Users/abhilashdhal/Desktop/cbioportal_gsoc/OmicsMLRepoHarmonizer


In [2]:
import gevent.monkey
gevent.monkey.patch_all(thread=False, select=False)

True

In [3]:
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
import os
import pandas as pd
from importlib import reload
import src.models.ontology_mapper_st as om_st
import src.Engine.ontology_mapping_engine as ome
import src.models.ontology_mapper_lm as om_lm
reload(om_st)
reload(ome)
reload(om_lm)

/opt/homebrew/anaconda3/envs/‘gsoc_harmon_2024’/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/‘gsoc_harmon_2024’/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abhilashdhal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abhilashdhal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
USER_AGENT environment variable not set, consider setting it to identify your requests.
[nltk_data] Downloading package 

<module 'src.models.ontology_mapper_lm' from '/Users/abhilashdhal/Desktop/cbioportal_gsoc/OmicsMLRepoHarmonizer/src/models/ontology_mapper_lm.py'>

## Load the data

In [4]:
cura_map = pd.read_csv('./data/cBioPortal_treatment_name_map.csv')
query_list = cura_map['original_value'].tolist()
small_corpus_list = cura_map['curated_ontology'].to_list()
# large_corpus_list = pd.read_csv('./data/corpus_from_NCIT_C1909.csv')

In [5]:
cura_map 

,original_value,curated_ontology,curated_ontology_term_id,curated_ontology_term_db
0,17-DMAG,Alvespimycin,NCIT:C38142,NCIT
1,19D12,Robatumumab,NCIT:C71527,NCIT
2,5 fu,Fluorouracil,NCIT:C505,NCIT
3,5-FU,Fluorouracil,NCIT:C505,NCIT
4,5-fluorouracil,Fluorouracil,NCIT:C505,NCIT
...,...,...,...,...
913,vincristine,Vincristine,NCIT:C933,NCIT
914,vincristine sulfate,Vincristine Sulfate,NCIT:C1739,NCIT
915,wide local excision,Wide Local Excision,NCIT:C94441,NCIT
916,wire localization,Wire-Guided Localization,NCIT:C91832,NCIT


## Initialize the Engine

In [6]:
other_params = {"test_or_prod": "prod"}
onto_engine_large = ome.OntoMapEngine(method='sap-bert', topk=5,
query=query_list, corpus=small_corpus_list,
cura_map=None, 
yaml_path='./src/models/method_model.yaml', 
om_strategy='lm', **other_params)

09/09//2024 12:21:1725864700 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module


## Run the Ontology Mapping

In [7]:
results_engine_testing = onto_engine_large.run()

09/09//2024 12:21:1725864702 PM - INFO - OntoMapEngine: Running Ontology Mapping
09/09//2024 12:21:1725864702 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
09/09//2024 12:21:1725864702 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
09/09//2024 12:21:1725864702 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module
09/09//2024 12:21:1725864702 PM - INFO - OntoMapLM.get_match_results: Creating embeddings for query_list and corpus


/opt/homebrew/anaconda3/envs/‘gsoc_harmon_2024’/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/homebrew/anaconda3/envs/‘gsoc_harmon_2024’/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


09/09//2024 12:22:1725864721 PM - INFO - OntoMapLM.get_match_results: Calculating cosine similarity matrix
09/09//2024 12:22:1725864721 PM - INFO - OntoMapLM.get_match_results: Generating results table
09/09//2024 12:22:1725864722 PM - INFO - OntoMapLM.get_match_results: Results Generated


## Display Results

In [12]:
results_engine_testing

(['Abiraterone',
  'Adalimumab',
  'Afatinib',
  'Alisertib',
  'Alpelisib',
  'Anastomosis',
  'Anterior Pelvic Exenteration',
  'Arsenic Trioxide',
  'Aspiration',
  'Atezolizumab',
  'Autopsy',
  'Axitinib',
  'Azacitidine',
  'Azathioprine',
  'Bevacizumab',
  'Bicalutamide',
  'Bilateral Mastectomy',
  'Bilateral Salpingectomy',
  'Biochemotherapy',
  'Biopsy',
  'Bleomycin',
  'Bortezomib',
  'Buparlisib',
  'Busulfan',
  'Cabazitaxel',
  'Cabozantinib',
  'Capecitabine',
  'Carboplatin',
  'Carmustine',
  'Cediranib',
  'Cetuximab',
  'Cilengitide',
  'Cisplatin',
  'Cladribine',
  'Clofarabine',
  'Cobimetinib',
  'Craniotomy',
  'Crenolanib',
  'Cyclophosphamide',
  'Cystectomy',
  'Cytarabine',
  'Cytosine',
  'Dabrafenib',
  'Dacomitinib',
  'Daratumumab',
  'Dasatinib',
  'Daunorubicin',
  'Decitabine',
  'Dexamethasone',
  'Docetaxel',
  'Donor Lymphocyte Infusion',
  'Doxorubicin',
  'Doxorubicin Hydrochloride',
  'Durvalumab',
  'Dutasteride',
  'Enasidenib',
  'Entosple